In [17]:
import pandas as pd 
import requests
import json


In [5]:
API_key = "5786cd713992d89f50a96d6c7495748f"
id_core = "MPDS2020b"
id_metadata = "2020-2"

## Data core consists of all the features created. Let's first access every single document they coded.

In [11]:
data_core = requests.get("https://manifesto-project.wzb.eu/api/v1/get_core?key=MPDS2020b&api_key={}".format(API_key)).text

In [25]:
df_core = pd.read_json(data_core)
df_core.columns = df_core.iloc[0]
df_core.drop(0, inplace=True)

We now restrict to Canada, United States, and United Kingdom

In [35]:
english_core = df_core[df_core['countryname'].isin(['Canada','United States','United Kingdom'])]


In [44]:
english_core['key'] = english_core['party'] + "_" + english_core['date']

<ipython-input-44-1593b6745c26>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_core['key'] = english_core['party'] + "_" + english_core['date']


In [149]:
english_core.to_csv("english_core.csv",index=False)

Here I create one long string that I'll later add to the get request. 

In [51]:
key_string = ""
keys = english_core['key'].values
for i, key in enumerate(keys):
    if i == len(keys) - 1:
        key_string += "keys[]=" + key
    else:
        key_string += "keys[]=" + key + "&"
    

## We need to now access the metadata

In [52]:
english_metadata = requests.get("https://manifesto-project.wzb.eu/api/v1/metadata?{}&api_key={}&version=2020-2".format(key_string, API_key)).text

In [ ]:
df_metadata = pd.DataFrame()
metadata = json.loads(english_metadata)['items']

for k in metadata:
    df_metadata = df_metadata.append(pd.DataFrame.from_dict(k,orient='index').T)

In [ ]:
df_metadata['key'] = df_metadata['party_id'].astype(str) + "_" + df_metadata['election_date'].astype(str)

In [151]:
df_metadata.to_csv("english_metadata.csv", index=False)

## Now we can finally get the text

In [59]:
english_text = requests.get("https://manifesto-project.wzb.eu/api/v1/texts_and_annotations?{}&api_key={}&version=2020-2".format(key_string, API_key)).text

In [63]:
english_text_json = json.loads(english_text)

In [108]:
dic = {}
for j in english_text_json['items']:
    string = ""
    for k in j['items']:
             string += k['text'] + " "
    dic.update({j['key']:string})

In [113]:
text_data = pd.DataFrame.from_dict(dic, orient='index', columns = ['text'])

In [153]:
text_data.reset_index(inplace = True)
text_data.columns = ['key','text']

In [155]:
text_data.to_csv("english_text.csv", index=False)